In [1]:
import logging
logging.basicConfig(format='\033[93m%(asctime)s %(levelname)s %(name)s:\033[0m %(message)s', level=logging.INFO, datefmt='%I:%M:%S')

# Load data

In [2]:
from quote_source.client import ServiceClient, Configuration, TimeFrameDTO
from quote_source.client.TimeFrame import TimeFrame
from datetime import date

In [3]:
cfg = Configuration()
cfg.host = "192.168.34.2:7002"
srvCNT = ServiceClient(cfg)

In [4]:
base_tf = TimeFrame.HOUR
base_tf

<TimeFrame.HOUR: 7>

In [5]:
markets = srvCNT.get_markets()
market = [m for m in markets if "Forex" in m.name][0]
market

{'id': 1, 'name': 'Forex'}

In [6]:
stocks = srvCNT.get_stock(market, "EURUSD")
stock = stocks[0]
stock

{'code': 'EURUSD', 'id': 11, 'market_id': 1, 'name': 'Eur/Usd'}

In [7]:
from_dt = date(2016,1,1)
till_dt = date(2021,1,1)

In [8]:
candles = srvCNT.get_quote(stock,base_tf, from_dt,till_dt)

In [9]:
import pandas as pd
df = pd.DataFrame([cndl.to_dict() for cndl in candles ])
df = df.set_index(pd.to_datetime(df["start_date_time"])).drop("start_date_time", axis = 1).rename({"close":"C","open":"O","high":"H","low":"L","volume":"V"},axis=1)
df

,O,H,L,C,V
start_date_time,,,,,
2020-02-02 21:00:00+00:00,1.10846,1.10901,1.10810,1.10833,76.0
2020-02-02 22:00:00+00:00,1.10820,1.10839,1.10810,1.10834,5.0
2020-02-02 23:00:00+00:00,1.10829,1.10854,1.10800,1.10820,56.0
2020-02-03 00:00:00+00:00,1.10820,1.10950,1.10790,1.10940,216.0
2020-02-03 01:00:00+00:00,1.10936,1.10950,1.10788,1.10849,85073.0
...,...,...,...,...,...
2021-01-01 19:00:00+00:00,1.22138,1.22138,1.22138,1.22138,720.0
2021-01-01 20:00:00+00:00,1.22138,1.22138,1.22138,1.22138,720.0
2021-01-01 21:00:00+00:00,1.22138,1.22138,1.22138,1.22138,720.0


# Проверяем, что можно поднять TimeFrame

In [10]:
def check_tf_could_be_up(from_tf:TimeFrame, to_tf:TimeFrame)->bool:
    return to_tf.to_seconds()%from_tf.to_seconds()==0

In [11]:
print(f"Успешное поднятие TF из {TimeFrame.HOUR} в {TimeFrame.DAY}: {check_tf_could_be_up(TimeFrame.H, TimeFrame.D)}")

Успешное поднятие TF из TimeFrame.HOUR в TimeFrame.DAY: True


# Поднимаем данные

In [12]:
def count_candle_in_up_candle(from_tf:TimeFrame, to_tf:TimeFrame)->int:
    return to_tf.to_seconds()//from_tf.to_seconds()

In [13]:
print(f"TimeFrame {TimeFrame.D} содержит {count_candle_in_up_candle(TimeFrame.H, TimeFrame.D)} candle {TimeFrame.H}")

TimeFrame TimeFrame.DAY содержит 24 candle TimeFrame.HOUR


In [22]:
import pandas as pd
from typing import Tuple
from datetime import datetime
from __future__ import annotations
from src.candle_up_time_frame.timeframe_candle_up.dt_compare import compare_strategy



class Condenser:
    def __init__(self, compare_dt_strategy = compare_strategy, col_map:colmap = colmap.get_short_map()) -> None:
        """Конструктор

        Args:
            compare_dt_strategy ([type], optional): Стратегия для получения функции определения начала новой свечи. Defaults to CompareStrategy.
            col_map (Condenser.colmap, optional): Маппер названий колонок и их назначение. Defaults to colmap.get_short_map().
        """
        self.logger = logging.getLogger("Condencer")
        self.__comp_dt_str = compare_dt_strategy
        self.__col_map = col_map
        
    def get_new_candle_flg(self, df: pd.DataFrame, upper_tf: TimeFrame)->pd.Series:
        comp_func = self.__comp_dt_str(upper_tf)
        self.logger.info("Get DT compare function")

        worker = new_candle_flag_builder(comp_func)
        
        return pd.Series(df.index.map(lambda idx: worker.get_flag(idx)),index=df.index)


In [18]:
condenser = Condenser()


In [21]:
condenser.get_new_candle_flg(df, TimeFrame.D)

03:41:59 INFO Condencer: Get DT compare function


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
target_df = df.apply(lambda row: condenser.check_base_row(row), axis=1)

DT
2016-01-01 00:00:00   2016-01-01 00:00:00
2016-01-01 01:00:00   2016-01-01 01:00:00
2016-01-01 02:00:00   2016-01-01 02:00:00
2016-01-01 03:00:00   2016-01-01 03:00:00
2016-01-01 04:00:00   2016-01-01 04:00:00
                              ...        
2021-01-01 20:00:00   2021-01-01 20:00:00
2021-01-01 21:00:00   2021-01-01 21:00:00
2021-01-01 22:00:00   2021-01-01 22:00:00
2021-01-01 23:00:00   2021-01-01 23:00:00
2021-01-02 00:00:00   2021-01-02 00:00:00
Length: 32180, dtype: datetime64[ns]

In [ ]:
import pandas as pd
data_df.apply(lambda row: pd.Series(row.name,row["C"]),axis=1)

NameError: name 'data_df' is not defined